In [20]:
import requests
import re
from bs4 import BeautifulSoup

In [21]:
def scrap_lbc(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Erreur {response.status_code} - L'URL semble invalide")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Titre
    title = soup.find('h1').text.strip() if soup.find('h1') else None

    # Description
    desc_tag = soup.find('div', {'data-qa-id': 'adview_description_container'})
    description = desc_tag.text.strip() if desc_tag else None

    # Localisation
    location_tag = soup.find('div', {'data-qa-id': 'adview_location_informations'})
    location = location_tag.text.strip() if location_tag else None

    # Collecte d'images avec filtre par ID unique
    image_dict = {}
    picture_tags = soup.find_all("picture")

    for picture in picture_tags:
        # sources
        for source in picture.find_all("source"):
            src = source.get("srcset", "")
            match = re.search(r'/images/.+?/([a-f0-9]{40})\.jpg', src)
            if match:
                image_id = match.group(1)
                if image_id not in image_dict:
                    image_dict[image_id] = src.split()[0]

        # img
        img = picture.find("img")
        if img and img.get("src"):
            src = img["src"]
            match = re.search(r'/images/.+?/([a-f0-9]{40})\.jpg', src)
            if match:
                image_id = match.group(1)
                if image_id not in image_dict:
                    image_dict[image_id] = src

    return {
        "title": title,
        "description": description,
        "location": location,
        "image_url": image_dict
    }


In [23]:
scrap_lbc('https://www.leboncoin.fr/ad/velos/2964205704')

{'title': 'LOOK 585, total LOOK',
 'description': "Vélo léger et solide, avec un beau cadre qui laisse paraître sa structure en carbone tressé... Le 585, est le premier cadre de la marque à moins d’un kilo ; de plus, voir sa fourche entièrement monocoque, voir le placement des nappes de carbone permettant d'augmenter la rigidité aux endroits nécessaires, ou au contraire de donner de la souplesse sur le tube de selle pour augmenter le confort. Equipé intégralement en Ultegra. Triple plateau pour les fins difficiles, K7 en 32. Cadre, selle, porte-…Voir plus",
 'location': None,
 'image_url': {'c93614c43d0d45ae248052fe37a296061f2918be': 'https://img.leboncoin.fr/api/v1/lbcpb1/images/c9/36/14/c93614c43d0d45ae248052fe37a296061f2918be.jpg?rule=classified-1200x800-webp',
  '8823a5d5a3ae4957dfde1c5cc081099fe5ccba63': 'https://img.leboncoin.fr/api/v1/lbcpb1/images/88/23/a5/8823a5d5a3ae4957dfde1c5cc081099fe5ccba63.jpg?rule=classified-1200x800-webp',
  'e6cadfd3623036fd3e117da0f00abdaf62b8a2a1': 